##### Saving words from newspaper titles in textfiles

In [4]:
import json

# Path to the JSON file
json_file_path = './cdx_results_2019.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# List to store the URLs
urls = []

# Loop through the data and extract 'url' values
for entry in data:
    if 'url' in entry:  # Check if 'url' key exists
        urls.append(entry['url'])  # Add the URL to the list

# Open a file to write the results
with open("titles.txt", "w") as file:
    # Loop through each URL
    for url in urls:
        # Remove 'https://' and split the URL at slashes
        parts = url.split('/')
        
        # Extract the last part after the slash, which usually contains the title
        title_with_params = parts[-1]
        
        # Remove any parameters (everything after '?')
        title = title_with_params.split('?')[0]
        
        # Replace hyphens with spaces to improve readability
        formatted_title = title.replace('-', ' ')
        
        # Write the formatted title to the file
        file.write(f"{formatted_title}\n")

print("Titles extracted and written to 'titles.txt'.")

# maybe store urls and words wich belong togethers in a dictionary

FileNotFoundError: [Errno 2] No such file or directory: './cdx_results_2019.json'

In [5]:
import json

# Function to split the URL into words from the title
def split_url_in_words(url):
    # Remove 'https://' and split the URL at slashes
    parts = url.split('/')
    
    # Extract the last part after the slash, which usually contains the title
    title_with_params = parts[-1]
    
    # Remove any parameters (everything after '?')
    title = title_with_params.split('?')[0]
    
    # Replace hyphens with spaces to improve readability
    formatted_title = title.replace('-', ' ')
    
    # Return the split words
    return formatted_title

# Path to the JSON file
json_file_path = './cdx_results_json_files/cdx_results_2019.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Empty dictionary to store the URLs as keys and the split title words as values
urls = {}

# Iterate through the JSON data
for entry in data:
    if 'url' in entry:
        #url = entry['url']  # Get the full URL
        #split_title = split_url_in_words(url)  # Split the URL title into words
        #urls[str(entry['url'])] = split_title  # Store the URL as key and split title as value
        urls[str(entry['url'])] = split_url_in_words(entry['url'])

# Writing dictionary to a textfile
with open('urls_titles.json', 'w') as output_file:
    json.dump(urls, output_file, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: './cdx_results_json_files/cdx_results_2019.json'

 Trying to scrape only the < title > elements in the links and look up if Chega is written as: 'Chega' or 'CHEGA':

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import time

# Path to the JSON file
file_name = 'cdx_results_2019.json'

# Open and load the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

titles = {}

# Loop through URLs; search for the title element in HTML-Script
for v in data:
    url = v['url']
    
    try:
        # Requesting the website and setting a timeout of 10 seconds
        response = requests.get(url, timeout=10, stream=True) # stream=True for downloading data in chunks not everything at once 
        
        # Delay between requests to avoid overloading the server
        time.sleep(1)  # 1-second delay

        # Initialize an empty content variable and stream content chunks
        html_content = b""
        
        # Stream the content until we find the closing </title> tag
        for chunk in response.iter_content(chunk_size=512):
            html_content += chunk
            if b"</title>" in html_content:
                break  # Stop streaming once the <title> tag is found

        # Parse only the partial content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Extract the <title> element
        title_tag = soup.find("title")

        # If no <title> tag is found, skip this page, makes program faster
        if not title_tag:
            continue

        # Extract the text from the <title> tag
        title_text = title_tag.get_text()

        # Check for "Chega" or "CHEGA" (case-sensitive check)
        if "Chega" in title_text or "CHEGA" or "Andre Ventura" in title_text:
            titles[url] = title_text  # Store the URL and title in the dictionary

    except requests.exceptions.Timeout:
        print(f"Timeout occurred for URL: {url}")
        continue  # Skip to the next URL if a timeout occurs

    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL: {url} with error: {e}")
        continue  # Skip to the next URL if another error occurs

# Writing the dictionary to a JSON file
with open('titles.json', 'w', encoding='utf-8') as json_file:
    json.dump(titles, json_file, indent=4, ensure_ascii=False)

# Print the results, only for debugging, to remove later
for url, title in titles.items():
    print(url, title)

In [ ]:
# Code in 1st delivery:

import json
import requests
from bs4 import BeautifulSoup
import time

# Path to the JSON file
file_name = 'cdx_results_2019.json'

# Open and load the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

titles = {}

# Loop through URLs; search for the title element in HTML-Script
counter = 0

for v in data:
    url = v['url']
    print(f'iteration {counter}')
    print(f'link: {url}')
    counter += 1
    
    try:
        # Requesting the website and setting a timeout of 10 seconds
        response = requests.get(url, timeout=10, stream=True) # stream=True for downloading data in chunks not everything at once 
        
        # Delay between requests to avoid overloading the server
        # time.sleep(1)  # 1-second delay; should may be in code if server gets too much requests, for now works fine without delay

        # Initialize an empty content variable and stream content chunks
        html_content = b""
        
        # Stream the content until we find the closing </title> tag
        for chunk in response.iter_content(chunk_size=512):
            html_content += chunk
            if b"</title>" in html_content:
                break  # Stop streaming once the <title> tag is found

        # Parse only the partial content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser") # optional lxml parser

        # Extract the <title> element
        title_tag = soup.find("title")

        # If no <title> tag is found, skip this page, makes program faster
        if not title_tag:
            continue

        # Extract the text from the <title> tag
        title_text = title_tag.get_text()

        # Check for "Chega" or "CHEGA" (case-sensitive check)
        if "Chega" in title_text or "CHEGA" or "Andre Ventura" in title_text:
            print(f'title text "{title_text}" is valid"')
            titles[url] = title_text  # Store the URL and title in the dictionary

    except requests.exceptions.Timeout:
        print(f"Timeout occurred for URL: {url}")
        continue  # Skip to the next URL if a timeout occurs

    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL: {url} with error: {e}")
        continue  # Skip to the next URL if another error occurs

# Writing the dictionary to a JSON file
with open('titles.json', 'w', encoding='utf-8') as json_file:
    json.dump(titles, json_file, indent=4, ensure_ascii=False)

In [ ]:
import pandas as pd

# Debug output:

for url, title in titles.items():
    print(f'{url}: {title}')

print(titles)

#df_titles = pd.DataFrame(data=titles)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all titles into one string
all_titles = ' '.join(titles.values())

# Generate a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_titles)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # No axes for a clean look
plt.show()

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import time

# Path to the JSON file
file_name = 'cdx_results_2019.json'

# Open and load the JSON file
with open(file_name, 'r') as file:
    data = json.load(file)

titles = {}

# Loop through URLs; search for the title element in HTML-Script
counter = 0

for v in data:
    url = v['url']
    print(f'iteration {counter}')
    print(f'link: {url}')
    counter += 1
    
    try:
        # Requesting the website and setting a timeout of 10 seconds
        response = requests.get(url, timeout=10, stream=True) # stream=True for downloading data in chunks not everything at once 
        
        # Delay between requests to avoid overloading the server
        #time.sleep(1)  # 1-second delay

        # Initialize an empty content variable and stream content chunks
        html_content = b""
        
        # Stream the content until we find the closing </title> tag
        for chunk in response.iter_content(chunk_size=512):
            html_content += chunk
            if b"</title>" in html_content:
                break  # Stop streaming once the <title> tag is found

        # Parse only the partial content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Extract the <title> element
        title_tag = soup.find("title")

        # If no <title> tag is found, skip this page, makes program faster
        if not title_tag:
            continue

        # Extract the text from the <title> tag
        title_text = title_tag.get_text()

        # Check for "Chega" or "CHEGA" (case-sensitive check)
        if "Chega" in title_text or "CHEGA" in title_text:
            print(f'title text "{title_text} is valid"')
            titles[url] = title_text  # Store the URL and title in the dictionary

    except requests.exceptions.Timeout:
        print(f"Timeout occurred for URL: {url}")
        continue  # Skip to the next URL if a timeout occurs


    except requests.exceptions.RequestException as e:
        print(f"Request failed for URL: {url} with error: {e}")
        continue  # Skip to the next URL if another error occurs

# Writing the dictionary to a JSON file
with open('titles.json', 'w', encoding='utf-8') as json_file:
    json.dump(titsles, json_file, indent=4, ensure_ascii=False)

In [ ]:
# extracting the text too